<a href="https://colab.research.google.com/github/zehavitc/NLP_Hw1/blob/master/NLP_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!ls '/content/drive/My Drive/NLP/HW1'

en.csv	fr.csv	it.csv	out.txt  tl.csv
es.csv	in.csv	nl.csv	pt.csv	 ZehavitTextTest.csv


In [0]:
import numpy as np
from collections import Counter, defaultdict
import csv
import math

g_startTag = "<s>"
g_endTag = "</s>"

def _update_grams(text, n_gram, n_counts): 
    #print(f'n_gram is {n_gram}')
    text = text.replace('\n', ' ')
    for i in range(0, len(text) - n_gram + 1):
      #print(f'i is {i}')
      t = text[i:i+n_gram-1]
      #print(t)
      n_char = text[i+n_gram-1]
      #print(n_char)
      n_counts[t][n_char] += 1
    #handle start and end of tweet 
    if n_gram == 1:
      n_counts[''][g_startTag] += 1
      n_counts[''][g_endTag] += 1
    if n_gram == 2:
      n_counts[g_startTag][text[0]] += 1
      n_counts[text[len(text)-1]][g_endTag] += 1 
    if n_gram > 2:
      n_counts[f'{g_startTag}{text[0]}'][text[n_gram -2]] += 1
      n_counts[f'{text[len(text) - n_gram + 1: len(text)]}'][g_endTag] += 1
      
def _output_model_to_file(n_counts, model_file_path):
    with open(model_file_path, "w") as model_file:
      counter_1gram = n_counts[0]['']
      counter_2gram = n_counts[1]
      counter_3gram = n_counts[2]
      
      #output 3-gram 
      print(f"3-grams:", file=model_file)
      trigramKeys = list(counter_3gram.keys())
      for tk in trigramKeys:
        #print(f'tk is: {tk}, len of tk is {len(tk)}')
        trigramValues = list(counter_3gram[tk].keys())
        if tk.startswith(g_startTag):
          firstChar = g_startTag
          secondChar = tk[3]
        elif tk.endswith(g_endTag):
          firstChar = tk[0]
          secondChar = g_endTag
        else:
          firstChar = tk[0]
          secondChar = tk[1]
        charsCount = counter_2gram[firstChar][secondChar]
        #print(f'chars count of {firstChar} {secondChar} is {charsCount}')
        for v in trigramValues:
          #print(f'v is: {v}')
          print(f'<{firstChar}><{secondChar}><{v}>\t{counter_3gram[tk][v]/charsCount}', file=model_file)
          #print(f'<{firstChar}><{secondChar}><{v}>\t{math.log(counter_3gram[tk][v]/charsCount, 2)}', file=model_file)
      print("", file=model_file)    
      
      #output 2-gram
      print(f"2-grams:", file=model_file)
      bigramKeys = list(counter_2gram.keys())
      for bk in bigramKeys:
        bigramValues = list(counter_2gram[bk].keys())
        charsCount = counter_1gram[bk]
        for v in  bigramValues:
          print(f'<{bk}><{v}>\t{counter_2gram[bk][v]/charsCount}', file=model_file) 
          #print(f'<{bk}><{v}>\t{math.log(counter_2gram[bk][v]/charsCount, 2)}', file=model_file) 
      print("", file=model_file)     
      
      #output 1-gram
      print(f"1-grams:", file=model_file)
      unigramKeys = list(counter_1gram.keys())
      charsCount = sum(list(counter_1gram.values()))
      for k in unigramKeys:
        print(f'<{k}>\t{counter_1gram[k]/charsCount}', file=model_file)
        #print(f'<{k}>\t{math.log(counter_1gram[k]/charsCount, 2)}', file=model_file)
      print("", file=model_file)




# In: corpus_file_path: a csv corpus file path - each line is a <TweetID>,<TweetText> 
# Out: model_file_path: model_file 
def	lm(corpus_file_path,	model_file_path):
  with open(corpus_file_path, 'r') as corpus_file:
    reader = csv.DictReader(corpus_file)
    #unigram, bigram, trigram
    n_counts = [defaultdict(Counter), defaultdict(Counter), defaultdict(Counter)]
    for row in reader:
      tweet_text = row["tweet_text"]   
      #print(tweet_text)
      for i in range(1,4):       
        _update_grams(tweet_text, i, n_counts[i-1])  
      
    _output_model_to_file(n_counts, model_file_path)
  print('lm done')
  

In [0]:
import re
from collections import namedtuple
from collections import Counter, defaultdict
import math
import sys

bigram_key = namedtuple('bigram_key', ["first"])
trigram_key = namedtuple("trigram_key", ["first", "second"])



def eval(input_file, model_file, weights):
  trigram_model, bigram_model, unigram_model = parse(model_file)
  with open(input_file, 'r') as input_file:
    reader = csv.DictReader(input_file)
    #unigram, bigram, trigram
    total_len = 0
    p_interp_list = []
    for row in reader:
      tweet_text = row["tweet_text"].replace('\n', ' ')
      total_len += (len(tweet_text) + 2)
  #     print(tweet_text)    
      for i in range(len(tweet_text)):
        if i==0:
            p_interp =  (weights[1]*bigram_model[bigram_key(first = g_startTag)][tweet_text[i]]) + \
                        (weights[2]*unigram_model[tweet_text[i]])           
        elif i==1:
            p_interp = (weights[0]*trigram_model[trigram_key(first = g_startTag, second = tweet_text[i-1])][tweet_text[i]]) + \
                       (weights[1]*bigram_model[bigram_key(first = tweet_text[i-1])][tweet_text[i]]) + \
                       (weights[2]*unigram_model[tweet_text[i]]) 
        else:
          p_interp = (weights[0]*trigram_model[trigram_key(first = tweet_text[i-2], second = tweet_text[i-1])][tweet_text[i]]) + \
                     (weights[1]*bigram_model[bigram_key(first = tweet_text[i-1])][tweet_text[i]]) + \
                     (weights[2]*unigram_model[tweet_text[i]])     
        p_interp_list.append(float(p_interp))
      #handle closing tag, assuming no empty tweet
      p_trigram = trigram_model[trigram_key(first = g_startTag, second = tweet_text[len(tweet_text)-1])][g_endTag] if len(tweet_text) == 1 else trigram_model[trigram_key(first = tweet_text[len(tweet_text) - 2], second = tweet_text[len(tweet_text)-1])][g_endTag]
      p_interp_closing = (weights[0]*p_trigram) + \
                   (weights[1]*(bigram_model[bigram_key(first = tweet_text[len(tweet_text)-1])][g_endTag])) + \
                   (weights[2]*(unigram_model[g_endTag])) 
      p_interp_list.append(float(p_interp_closing))
    #calc entropy 
    log_sum = sum([(math.log(v,2) if v != 0  else sys.float_info.min) for v in p_interp_list])
    entropy = -log_sum / total_len
    return math.pow(2,entropy)
    
  
def parse(model_file):
  with open(model_file) as model:
    # Skips text before the beginning of the interesting block:
    for line in model:
        if line.strip() == '3-grams:':  # Or whatever test is needed
            break
    # Reads text until the end of the block:
    trigram_model = defaultdict(Counter)

    for line in model:  # This keeps reading the file
        if line.strip() == '':
            break
        #print(line)  # Line is extracted (or block_of_lines.append(line), etc.)
        match = re.search("<(.*)><(.*)><(.*)>\t(.*)", line)
        #print(f'grpup 1 is: {match.group(1)}')
        #print(f'grpup 2 is: {match.group(2)}')
        #print(line)
        k = trigram_key(first=match.group(1), second=match.group(2))
        #print(f'key is {k}')
        trigram_model[k][match.group(3)] = float(match.group(4))
    
    for line in model:
      if line.strip() =='2-grams:':
        break;
    bigram_model = defaultdict(Counter)
    for line in model:  # This keeps reading the file
      if line.strip() == '':
          break
#       print(line)  # Line is extracted (or block_of_lines.append(line), etc.)
      match = re.search("<(.*)><(.*)>\t(.*)", line)
      #print(f'grpup 1 is: {match.group(1)}')
      #print(f'grpup 2 is: {match.group(2)}')
      k = bigram_key(first=match.group(1))
      #print(f'key is {k}')
      bigram_model[k][match.group(2)] = float(match.group(3))

    # print(bigram_model)
    
    for line in model:
      if line.strip() =='1-grams:':
#         print(line)
        break;
    unigram_model = defaultdict(float)
    for line in model:  # This keeps reading the file
      if line.strip() == '':
          break
#       print(line)  # Line is extracted (or block_of_lines.append(line), etc.)
      match = re.search("<(.*)>\t(.*)", line)
      #print(f'grpup 1 is: {match.group(1)}')
      #print(f'grpup 2 is: {match.group(2)}')
      #print(f'key is {k}')
      unigram_model[match.group(1)] = float(match.group(2))

    #print(unigram_model)
    return (trigram_model, bigram_model, unigram_model)
 

In [0]:
!rm '/content/drive/My Drive/NLP/HW1/out.txt'

In [37]:
#corpus_file_path = '/content/drive/My Drive/NLP/HW1/en.csv'
#model_file_path =  '/content/drive/My Drive/NLP/HW1/out.txt'
#lm(corpus_file_path, model_file_path)

lm done


In [64]:
#eval('/content/drive/My Drive/NLP/HW1/es.csv', '/content/drive/My Drive/NLP/HW1/out.txt', [0.4,0.3,0.3])

20.085902692255633

In [75]:
import pandas as pd 


def _get_path_for_name(name):
  return f'/content/drive/My Drive/NLP/HW1/{name}.csv'

langs  = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
#langs  = ['en', 'es']
files = [_get_path_for_name(l) for l in langs]
weights = [0.4,0.3,0.3]

def identify_lang():
  #results = [langs.copy()]
  #results[0].insert(0, ' ')
  results = []
  
  for l in langs:
    df = pd.read_csv(_get_path_for_name(l), encoding='ISO-8859-1')
    train = df.sample(frac=0.9)
    train_path = _get_path_for_name(f'{l}_train.csv')
    model_path = _get_path_for_name(f'{l}_model.txt')
    df.to_csv(train_path)
    lm(train_path, model_path)
    lang_results = []
    for l in langs:
      lang_results.append(eval(_get_path_for_name(l), model_path, weights))
    results.append(lang_results)
    
  return pd.DataFrame(results, langs, langs)

identify_lang()

lm done
lm done
lm done
lm done
lm done
lm done
lm done
lm done


,en,es,fr,in,it,nl,pt,tl
en,12.098403,19.856888,20.710524,21.527680,18.873814,19.650246,21.405984,19.754943
es,20.280610,12.258516,18.329501,23.174260,16.846648,21.814565,16.906084,21.802658
fr,18.971111,17.475795,13.762360,23.185811,18.488773,20.702984,19.714554,22.491690
in,19.568051,20.673424,24.112045,12.776977,20.395312,21.110217,22.342062,18.037316
it,19.911945,17.150583,18.068464,23.172832,11.570243,22.167298,18.074792,20.455094
nl,18.163060,19.627426,19.137466,21.538902,20.282337,12.348462,21.182268,20.649026
pt,20.703215,16.216116,18.857026,23.698860,17.468701,22.737250,12.255169,21.376160
tl,18.016843,19.488982,23.998614,18.694047,19.571747,21.809845,21.089908,11.420352
